In [1]:
# 前提１：　EFファイル　＝　DPC包括　+　出来高（DPC期間超　+　DPC適用区分==0）
# 前提２：　Dファイル　＝　出来高（DPC期間超　+　DPC適用区分==0） + DPC入院料

# Making D file from EF file and FF1 file

# Step 1. Aggregate table dpc_ef using table tensumaster
# Step 2. Make "DPC score table" from table dpc_ff1 and table dpc master
# Step 3. Concat Step 1's table and Step 2's table and sort by patient's id and "実施年月日"

In [1]:
import pandas as pd
import psycopg2 as pg
import pandas.io.sql as psql
import datetime
#import numba
#import dask.dataframe as dd

In [2]:
# Variables
start = "2017-01-01"
end = "2017-01-31"

month = "1701"

nengo = "h28"

#医療機関係数
cof = 1.2800

#処理日
imp_date = "20170324"

In [3]:
# Step 0. Read dummy D file and get D file's column as list

D = pd.read_excel("dummyD.xlsx")

In [4]:
dcol = D.columns.tolist()

In [5]:
# Step 1. Aggregate table dpc_ef using table tensumaster

In [6]:
# Get dpc_ef as EF
with pg.connect(database='uonuma_kikan',user='postgres',host='localhost',port=5432,password=12345) as conn:
    sql = """SELECT * FROM dpc_ef 
              --WHERE データ区分 <> 90
              --AND データ区分 <> 92
              --AND データ区分 <> 97
              --AND データ区分 NOT BETWEEN 20 AND 39
                WHERE 実施年月日 BETWEEN '{0}' AND '{1}';""".format(start,end)
    EF = psql.read_sql(sql,conn)

In [8]:
# Get tensumaster as TEN
with pg.connect(database='uonuma_kikan',user='postgres',host='localhost',port=5432,password=12345) as conn:
    sql = "SELECT * FROM tensumaster_{0};".format(nengo)
    TEN = psql.read_sql(sql,conn)

In [10]:
TEN = TEN[["診療行為コード","dpc適用区分"]]

In [11]:
EFTEN = pd.merge(EF,TEN,left_on="レセプト電算コード",right_on="診療行為コード",how="left")

In [12]:
EFTEN["行為明細区分情報"] = EFTEN["行為明細区分情報"].fillna("")

In [13]:
EFTEN.shape

(335108, 34)

In [14]:
# ｄｐｃ入院料レコード作成のためのフルEFTENをここで保存しておく
full = EFTEN

In [1]:
#EFTEN[EFTEN["行為明細区分情報"]=='100000000000']

In [16]:
set(EFTEN["行為明細区分情報"])

{'',
 '000000000000',
 '000010000000',
 '002100000000',
 '002200000000',
 '010000000000',
 '010010000000',
 '100000000000'}

In [20]:
# 3月２２日追加

# dpc適用区分の見直し
    # 包括対象：　0（包括評価対象）
    # 包括=出来高中間（a）：　1（出来高部分で算定可能な診療行為）　
    # 出来高:9（包括評価対象外）
    # dpc適用区分が空欄（ｂ）
    
# 空欄(b)より退院時処方をピックアップ
    # 退院時処方分：kuran_taiin
    # 退院時処方以外：kuran_remove(b')
    
# 包括=出来高中間(a),kuran_remove(b')のデータ区分の見直し
    #　出来高診療区分：10~19, 50~59, (80~89) 

    
###### 退院時処方の行為明細区分情報が正しいか（Fファイル時代に行為明細区分情報の１桁目が"1"になっていたものが、EFファイルになると消えていないか）

In [21]:
# dpc適用区分の見直し

In [22]:
# 包括対象：　0（包括評価対象）
# ここには退院時処方は入っていない
hokat = EFTEN[EFTEN["dpc適用区分"]==0]

In [48]:
#hokat.to_excel("test_hokat_{}.xlsx".format(imp_date))

In [23]:
# 包括=出来高中間(a)：　1（出来高部分で算定可能な診療行為）
# ここには退院時処方は入っていない
medium = EFTEN[EFTEN["dpc適用区分"]==1]

In [51]:
#medium.to_excel("test_medium_{}.xlsx".format(imp_date))

In [24]:
# 出来高:9（包括評価対象外）
deki9 = EFTEN[EFTEN["dpc適用区分"]==9]

In [25]:
deki9

,施設コード,データ識別番号,退院年月日,入院年月日,データ区分,順序番号,行為明細番号,病院点数マスタコード,レセプト電算コード,解釈番号,...,レセプト科区分,診療科区分,医師コード,病棟コード,病棟区分,入外区分,施設タイプ,id,診療行為コード,dpc適用区分


In [26]:
# dpc適用区分が空欄(b)
# ここに退院時処方入ってる
kuran = EFTEN[EFTEN["dpc適用区分"].isnull()]

In [63]:
#kuran.to_excel("test_kuran_{}.xlsx".format(imp_date))

In [ ]:
# 空欄(b)より退院時処方をピックアップ
　　　　# 退院時処方分：kuran_taiin
    # 退院時処方以外：kuran_remove(b')

In [27]:
# 退院時処方のみ抽出（→出来高としてDファイルに格納）
kuran_taiin = kuran[kuran["行為明細区分情報"].apply(lambda x:x[0] if x != "" else x)=="1"]

In [28]:
# 退院時処方をのぞいたもの(→包括)
kuran_remove = kuran[kuran["行為明細区分情報"].apply(lambda x:x[0] if x != "" else x)!="1"]

In [ ]:
# 包括=出来高中間(a),kuran_remove(b')のデータ区分の見直し
    #　出来高診療区分：10~19, 50~59, (80~89) 

In [29]:
set(medium["データ区分"])

{11.0, 13.0, 14.0, 33.0, 40.0, 50.0, 54.0, 60.0, 70.0, 80.0, 90.0, 92.0, 97.0}

In [30]:
[x for x in range(10,20)]

[10, 11, 12, 13, 14, 15, 16, 17, 18, 19]

In [31]:
# 中間帯の内、データ区分から判断して出来高になるもの
medium_deki = medium[(medium["データ区分"].isin(range(10,20)))|(medium["データ区分"].isin(range(50,60)))|(medium["データ区分"].isin(range(80,90)))]

In [32]:
# 中間帯の内、データ区分から判断して包括になるもの
medium_hokat = medium[~((medium["データ区分"].isin(range(10,20)))|(medium["データ区分"].isin(range(50,60)))|(medium["データ区分"].isin(range(80,90))))]

In [33]:
# 空欄（退院時処方除く）の内、データ区分から判断して出来高になるもの
kr_deki = kuran_remove[(kuran_remove["データ区分"].isin(range(10,20)))|(kuran_remove["データ区分"].isin(range(50,60)))|(kuran_remove["データ区分"].isin(range(80,90)))]

In [34]:
# 空欄（退院時処方除く）の内、データ区分から判断して包括になるもの
kr_hokat = kuran_remove[~((kuran_remove["データ区分"].isin(range(10,20)))|(kuran_remove["データ区分"].isin(range(50,60)))|(kuran_remove["データ区分"].isin(range(80,90))))]

In [35]:
# 出来高ファイル（medium_deki, deki9, kuran_taiin, kr_deki）のconcat

eften = pd.concat([medium_deki,deki9])
eften = pd.concat([eften,kuran_taiin])
eften = pd.concat([eften,kr_deki])

In [36]:
eften.reset_index(inplace=True)

In [37]:
eften.drop("index",1,inplace=True)

In [38]:
# 包括ファイル(hokat, medium_hokat, kr_hokat)のconcat

EFTEN = pd.concat([hokat,medium_hokat])
EFTEN = pd.concat([EFTEN,kr_hokat])

In [39]:
EFTEN.reset_index(inplace=True)

In [40]:
EFTEN.drop("index",1,inplace=True)

In [41]:
print(full.shape,eften.shape,EFTEN.shape)

(335108, 34) (22711, 34) (312397, 34)


In [15]:
'''
# 退院時処方のみ抽出（→出来高としてDファイルに格納）
taiin = EFTEN[EFTEN["行為明細区分情報"].apply(lambda x:x[0] if x != "" else x)=="1"]
'''

In [16]:
'''
# 退院時処方をのぞいたもの
exEFTEN = EFTEN[EFTEN["行為明細区分情報"].apply(lambda x:x[0] if x != "" else x)!="1"]
'''

In [17]:
'''
# exEFTENからdpc適用外のみ抽出（→出来高としてDファイルに格納）
eften = exEFTEN[exEFTEN["dpc適用区分"]==0]
'''

In [18]:
'''
# exEFTENからdpc適用のみ抽出（→後でその内のdpc期間外を抽出しDファイルに格納）
EFTEN = exEFTEN[exEFTEN["dpc適用区分"]==1]
'''

In [42]:
# Choose the column which D file has
#taiin = pd.DataFrame(taiin,columns=dcol)
full = pd.DataFrame(full,columns=dcol)
eften = pd.DataFrame(eften,columns=dcol)
EFTEN = pd.DataFrame(EFTEN,columns=dcol)

In [37]:
#taiin

In [40]:
#taiin[~(taiin["分類番号"].isnull())]

In [46]:
full[~(full["分類番号"].isnull())]

,施設コード,データ識別番号,退院年月日,入院年月日,データ区分,順序番号,病院点数マスタコード,レセプト電算コード,解釈番号,診療明細名称,...,医師コード,病棟コード,病棟区分,入外区分,施設タイプ,算定開始日,算定終了日,算定起算日,分類番号,医療機関係数


In [47]:
eften[~(eften["分類番号"].isnull())]

,施設コード,データ識別番号,退院年月日,入院年月日,データ区分,順序番号,病院点数マスタコード,レセプト電算コード,解釈番号,診療明細名称,...,医師コード,病棟コード,病棟区分,入外区分,施設タイプ,算定開始日,算定終了日,算定起算日,分類番号,医療機関係数


In [48]:
EFTEN[~(EFTEN["分類番号"].isnull())]

,施設コード,データ識別番号,退院年月日,入院年月日,データ区分,順序番号,病院点数マスタコード,レセプト電算コード,解釈番号,診療明細名称,...,医師コード,病棟コード,病棟区分,入外区分,施設タイプ,算定開始日,算定終了日,算定起算日,分類番号,医療機関係数


In [29]:
#taiin.to_excel("test_taiin_{0}.xlsx".format(imp_date))

In [ ]:
full.to_excel("test_full_{0}.xlsx".format(imp_date))

In [30]:
eften.to_excel("test_eften_dpc0_{0}.xlsx".format(imp_date))

In [31]:
EFTEN.to_excel("test_EFTEN_dpc1_{0}.xlsx".format(imp_date))

In [19]:
# Step 2. Make "DPC score table" from table dpc_ff1 and table dpcmaster

In [49]:
df_ef = EF

In [50]:
df_ef = df_ef[["データ識別番号","入院年月日","実施年月日"]]

In [51]:
df_ef["在院日数"] = df_ef["実施年月日"] - df_ef["入院年月日"] + datetime.timedelta(days=1)

C:\Program Files\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [52]:
df_ef.shape

(335108, 4)

In [53]:
df_ef.drop_duplicates(["データ識別番号","入院年月日","実施年月日"],inplace=True)

C:\Program Files\Anaconda3\lib\site-packages\pandas\util\decorators.py:91: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return func(*args, **kwargs)


In [54]:
df_ef.shape

(8127, 4)

In [55]:
df_ef["key"] = df_ef["データ識別番号"].apply(lambda x:str(x)) +"_"+ df_ef["入院年月日"].apply(lambda x:str(x))

C:\Program Files\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [56]:
with pg.connect(database='uonuma_kikan',user='postgres',host='localhost',port=5432,password=12345) as conn:
    sql = """
SELECT データ識別番号
       ,入院年月日
  FROM dpc_ff1_{0};""".format(month)
    df_ff = psql.read_sql(sql,conn)

In [57]:
df_ff["key"] = df_ff["データ識別番号"].apply(lambda x:str(x)) +"_"+ df_ff["入院年月日"].apply(lambda x:str(x))

In [58]:
df_ef = df_ef[df_ef["key"].isin(df_ff["key"])]

In [59]:
df_ef.shape

(4530, 5)

In [61]:
with pg.connect(database='uonuma_kikan',user='postgres',host='localhost',port=5432,password=12345) as conn:
    sql = """
SELECT 患者番号
       ,dpcコード
       ,入院日（転入日）
  FROM dpc_order
  WHERE 入院日（転入日） BETWEEN '{0}' AND '{1}';""".format(start,end)
    dpc_odr = psql.read_sql(sql,conn)

In [62]:
dpc_odr["データ識別番号"] = dpc_odr["患者番号"]+10

In [64]:
dpc_odr["key"] = dpc_odr["データ識別番号"].apply(lambda x:str(x)) +"_"+ dpc_odr["入院日（転入日）"].apply(lambda x:str(x))

In [65]:
d_odr = dpc_odr[["key","dpcコード"]]

In [66]:
df_ef.shape

(4530, 5)

In [67]:
d_odr.shape

(402, 2)

In [68]:
d_odr.drop_duplicates("key",inplace=True)

C:\Program Files\Anaconda3\lib\site-packages\pandas\util\decorators.py:91: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return func(*args, **kwargs)


In [69]:
d_odr.shape

(402, 2)

In [70]:
ef_odr = pd.merge(df_ef,d_odr,on="key",how='left')

In [71]:
ef_odr.shape

(4530, 6)

In [72]:
with pg.connect(database='uonuma_kikan',user='postgres',host='localhost',port=5432,password=12345) as conn:
    sql = """
SELECT 診断群分類番号
       ,入院期間Ⅰ（日）
       ,入院期間Ⅱ（日）
       ,入院期間Ⅲ（日）
       ,入院期間Ⅰ（点）
       ,入院期間Ⅱ（点）
       ,入院期間Ⅲ（点）
  FROM dpcmaster_{0};""".format(nengo)
    mstr = psql.read_sql(sql,conn)

In [74]:
ef_odr.rename(columns={"dpcコード":"診断群分類番号"},inplace=True)

In [75]:
mstr.head()

,診断群分類番号,入院期間Ⅰ（日）,入院期間Ⅱ（日）,入院期間Ⅲ（日）,入院期間Ⅰ（点）,入院期間Ⅱ（点）,入院期間Ⅲ（点）
0,010010xx99000x,3.0,9.0,30.0,3278.0,2022.0,1719.0
1,010010xx99001x,8.0,23.0,60.0,2667.0,2140.0,1819.0
2,010010xx9901xx,10.0,30.0,90.0,2804.0,2252.0,1914.0
3,010010xx9902xx,4.0,13.0,60.0,4855.0,2513.0,2053.0
4,010010xx99030x,7.0,17.0,60.0,2444.0,1903.0,1618.0


In [76]:
eo_ms = pd.merge(ef_odr,mstr,on="診断群分類番号",how='left')

In [72]:
# EFファイルからその日の出来高総点数を取得

In [78]:
deki = EF

In [79]:
deki = deki[["データ識別番号","入院年月日","実施年月日","出来高実績点数","行為回数"]]

In [80]:
deki["日別出来高総点数"] = deki["出来高実績点数"]*deki["行為回数"]

C:\Program Files\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [81]:
deki["jkey"] = deki["データ識別番号"].apply(lambda x:str(x)) +"_"+ deki["実施年月日"].apply(lambda x:str(x))

C:\Program Files\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [82]:
dk = pd.DataFrame(deki.groupby(["jkey"])["日別出来高総点数"].sum())

In [83]:
dk = dk.reset_index()

In [84]:
dk.head()

,jkey,日別出来高総点数
0,100006_2017-01-12,4309.0
1,100006_2017-01-13,4998.0
2,100006_2017-01-16,8.0
3,100006_2017-01-17,8.0
4,100006_2017-01-18,8.0


In [85]:
dk.shape

(8127, 2)

In [86]:
eo_ms["jkey"] = eo_ms["データ識別番号"].apply(lambda x:str(x))+"_"+eo_ms["実施年月日"].apply(lambda x:str(x))

In [87]:
eo_ms.shape

(4530, 13)

In [88]:
eom_j = pd.merge(eo_ms,dk,on="jkey",how='left')

In [90]:
eom_j["入院期間Ⅰ（日）"] = eom_j["入院期間Ⅰ（日）"].fillna(0)
eom_j["入院期間Ⅱ（日）"] = eom_j["入院期間Ⅱ（日）"].fillna(0)
eom_j["入院期間Ⅲ（日）"] = eom_j["入院期間Ⅲ（日）"].fillna(0)

In [91]:
eom_j["入院期間Ⅰ（点）"] = eom_j["入院期間Ⅰ（点）"].fillna(0)
eom_j["入院期間Ⅱ（点）"] = eom_j["入院期間Ⅱ（点）"].fillna(0)
eom_j["入院期間Ⅲ（点）"] = eom_j["入院期間Ⅲ（点）"].fillna(0)

In [92]:
eom_j["入院期間Ⅰ（日）"] = eom_j["入院期間Ⅰ（日）"].apply(lambda x:datetime.timedelta(days=x))
eom_j["入院期間Ⅱ（日）"] = eom_j["入院期間Ⅱ（日）"].apply(lambda x:datetime.timedelta(days=x))
eom_j["入院期間Ⅲ（日）"] = eom_j["入院期間Ⅲ（日）"].apply(lambda x:datetime.timedelta(days=x))

In [93]:
def kikanhelper(df):
    '''期間ヘルパー　在院日数を元にDPC入院期間カラムを作成します'''
    Zaiin = df['在院日数']
    Kikan1 = df['入院期間Ⅰ（日）']
    Kikan2 = df['入院期間Ⅱ（日）']
    Kikan3 = df['入院期間Ⅲ（日）']
    DPCKIKAN = []
    df["DPC期間"] = ""

    for zaiin,kikan1,kikan2,kikan3 in zip(Zaiin,Kikan1,Kikan2,Kikan3):
        if zaiin == datetime.timedelta(days=0):
            dpckikan = '出来高分岐'
        elif zaiin <= kikan1:
            dpckikan = '期間I'
        elif zaiin > kikan1 and zaiin <= kikan2:
            dpckikan = '期間II'
        elif zaiin > kikan2 and zaiin <= kikan3:
            dpckikan = '期間III'
        else:
            dpckikan = 'DPC期間超'
        DPCKIKAN.append(dpckikan)
        
    df["DPC期間"] = DPCKIKAN

In [94]:
def tensuhelper(df):
    '''点数ヘルパー　期間ヘルパーが出したDPC期間を元に、
    当該実施年月日のDPC点数を係数込みで返します'''
    DPCKIKAN = df["DPC期間"]
    Ten1 = df['入院期間Ⅰ（点）']
    Ten2 = df['入院期間Ⅱ（点）']
    Ten3 = df['入院期間Ⅲ（点）']
    Original = df['日別出来高総点数'] 
    '''↑これはサブパラメータ。元々は出来高データを元にしていたので、
    DPC期間超後は出来高点数が入るように工夫していた。この辺のロジックは頑張って自力実装するか、期間超は無視するコードにするしか無いかも'''
    DPCTEN = []
    df["dpc総点数"] = ""
    
    for dpckikan,ten1,ten2,ten3,original in zip(DPCKIKAN,Ten1,Ten2,Ten3,Original):
        if dpckikan == '期間I':
            dpcten = ten1 * cof
        elif dpckikan == '期間II':
            dpcten = ten2 * cof
        elif dpckikan == '期間III':
            dpcten = ten3 * cof
        else:
            dpcten = original
        DPCTEN.append(dpcten)
    
    df["dpc総点数"] = DPCTEN

In [95]:
kikanhelper(eom_j)

In [97]:
tensuhelper(eom_j)

In [77]:
# Step 3. Merge Step 1's table and Step 2's table and sort by patient's id and "実施年月日"

In [2]:
# Step 1's table
#print(full,EFTEN,eften)

In [100]:
EFTEN.columns

Index(['施設コード', 'データ識別番号', '退院年月日', '入院年月日', 'データ区分', '順序番号', '病院点数マスタコード',
       'レセプト電算コード', '解釈番号', '診療明細名称', '行為点数', '行為薬剤料', '行為材料料', '円点区分', '行為回数',
       '保険者番号', 'レセプト種別コード', '実施年月日', 'レセプト科区分', '診療科区分', '医師コード', '病棟コード',
       '病棟区分', '入外区分', '施設タイプ', '算定開始日', '算定終了日', '算定起算日', '分類番号', '医療機関係数'],
      dtype='object')

In [3]:
# Step 2's table
#eom_j

In [86]:
#eom_j.to_excel("test_eom_j.xlsx")

In [102]:
ksh = pd.DataFrame(eom_j.groupby("key")["実施年月日"].min())

In [103]:
ksh = ksh.reset_index()

In [104]:
ksh.rename(columns={"実施年月日":"算定開始日"},inplace=True)

In [105]:
ksh.head(2)

,key,算定開始日
0,100006_2017-01-12,2017-01-12
1,100996_2017-01-16,2017-01-16


In [106]:
shr = pd.DataFrame(eom_j.groupby("key")["実施年月日"].max())

In [107]:
shr = shr.reset_index()

In [108]:
shr.rename(columns={"実施年月日":"算定終了日"},inplace=True)

In [109]:
shr.head(2)

,key,算定終了日
0,100006_2017-01-12,2017-01-19
1,100996_2017-01-16,2017-01-18


In [110]:
full.shape

(335108, 30)

In [111]:
eften.shape

(22711, 30)

In [112]:
EFTEN.shape

(312397, 30)

In [113]:
#taiin["key"] = taiin["データ識別番号"].apply(lambda x:str(x)) +"_"+ taiin["入院年月日"].apply(lambda x:str(x))
full["key"] = full["データ識別番号"].apply(lambda x:str(x)) +"_"+ full["入院年月日"].apply(lambda x:str(x)) 
eften["key"] = eften["データ識別番号"].apply(lambda x:str(x)) +"_"+ eften["入院年月日"].apply(lambda x:str(x)) 
EFTEN["key"] = EFTEN["データ識別番号"].apply(lambda x:str(x)) +"_"+ EFTEN["入院年月日"].apply(lambda x:str(x)) 

In [171]:
#taiin

In [117]:
#taiin.drop(["算定開始日","算定終了日","算定起算日","分類番号"],1,inplace=True)
full.drop(["算定開始日","算定終了日","算定起算日","分類番号"],1,inplace=True)
eften.drop(["算定開始日","算定終了日","算定起算日","分類番号"],1,inplace=True)
EFTEN.drop(["算定開始日","算定終了日","算定起算日","分類番号"],1,inplace=True)

In [111]:
#taiin

In [121]:
#taiin = pd.merge(taiin,ksh,on="key",how='left')
full = pd.merge(full,ksh,on="key",how='left')
eften = pd.merge(eften,ksh,on="key",how='left')
EFTEN = pd.merge(EFTEN,ksh,on="key",how='left')

In [122]:
#taiin = pd.merge(taiin,shr,on="key",how='left')
full = pd.merge(full,shr,on="key",how='left')
eften = pd.merge(eften,shr,on="key",how='left')
EFTEN = pd.merge(EFTEN,shr,on="key",how='left')

In [123]:
#taiin["算定起算日"] = taiin["算定開始日"]
full["算定起算日"] = full["算定開始日"]
eften["算定起算日"] = eften["算定開始日"]
EFTEN["算定起算日"] = EFTEN["算定開始日"]

In [182]:
#taiin

In [127]:
bun = eom_j[["jkey","診断群分類番号"]]

In [128]:
bun.rename(columns={"診断群分類番号":"分類番号"},inplace=True)

C:\Program Files\Anaconda3\lib\site-packages\pandas\core\frame.py:2834: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  **kwargs)


In [129]:
bun = bun[~(bun["分類番号"].isnull())]

In [188]:
#taiin.shape

In [133]:
full.shape

(335108, 30)

In [130]:
eften.shape

(22711, 30)

In [131]:
EFTEN.shape

(312397, 30)

In [134]:
bun.shape

(2688, 2)

In [135]:
bun.drop_duplicates("jkey",inplace=True)

In [136]:
bun.shape

(2688, 2)

In [137]:
#taiin["jkey"] = taiin["データ識別番号"].apply(lambda x:str(x)) +"_"+ taiin["実施年月日"].apply(lambda x:str(x))
full["jkey"] = full["データ識別番号"].apply(lambda x:str(x)) +"_"+ full["実施年月日"].apply(lambda x:str(x))
eften["jkey"] = eften["データ識別番号"].apply(lambda x:str(x)) +"_"+ eften["実施年月日"].apply(lambda x:str(x))
EFTEN["jkey"] = EFTEN["データ識別番号"].apply(lambda x:str(x)) +"_"+ EFTEN["実施年月日"].apply(lambda x:str(x))

In [138]:
#tb = pd.merge(taiin,bun,on="jkey",how='left')
fl = pd.merge(full,bun,on="jkey",how='left')
eb = pd.merge(eften,bun,on="jkey",how='left')
EB = pd.merge(EFTEN,bun,on="jkey",how='left')

In [173]:
##tb.to_excel("test_tb.xlsx")
#eb.to_excel("tet_eb.xlsx")
#EB.to_excel("test_EB.xlsx")

In [196]:
#tb.shape

In [139]:
fl.shape

(335108, 32)

In [140]:
eb.shape

(22711, 32)

In [141]:
EB.shape

(312397, 32)

In [142]:
# 包括期間内のレコードを抽出
comp = eom_j[~(eom_j["DPC期間"] == "DPC期間超")]

In [144]:
cmplist = comp["jkey"]

In [146]:
# 包括期間内レコードは削除(期間外の出来高分のみ抽出)
EB = EB[~(EB["jkey"].isin(cmplist))]

In [147]:
EB.shape

(208147, 32)

In [138]:
#tb.shape

In [148]:
fl.shape

(335108, 32)

In [149]:
eb.shape

(22711, 32)

In [140]:
#eb = pd.concat([tb,eb])

In [150]:
EetBbb = pd.concat([EB,eb])

In [151]:
EetBbb.shape

(230858, 32)

In [152]:
EetBbb.reset_index(inplace=True)

In [153]:
EetBbb.drop("index",1,inplace=True)

In [4]:
#EetBbb
# EFファイルの内、出来高レコード(eften)と、包括レコード(EFTEN)のうちdpc期間外（出来高）をconcatしたもの

In [189]:
#　このあと、レセプト電算コード930000000となるレコード(包括期間内のレコード)をどっかから持ってきてEetBbbにconcat

In [155]:
#　診断群分類番号が入っているレコードのみ取得
dpc = eom_j[~(eom_j["診断群分類番号"].isnull())]

In [156]:
dpc["DPC入院料"] = ""

C:\Program Files\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [157]:
dpc = dpc.reset_index()

In [158]:
dpc = dpc.drop("index",1)

In [156]:
#ddpc = dd.from_pandas(dpc,2)

In [160]:
'''
# using dask dataframe
for i in ddpc.index:
    if ddpc.loc[i,"DPC期間"] == "期間I":
        ddpc.loc[i,"DPC入院料"] = ddpc.loc[i,"入院期間Ⅰ（点）"]
    elif ddpc.loc[i,"DPC期間"] == "期間II":
        ddpc.loc[i,"DPC入院料"] = ddpc.loc[i,"入院期間Ⅱ（点）"]
    elif ddpc.loc[i,"DPC期間"] == "期間III":
        ddpc.loc[i,"DPC入院料"] = ddpc.loc[i,"入院期間Ⅲ（点）"]
    else:
        pass
'''

'\n# using dask dataframe\nfor i in ddpc.index:\n    if ddpc.loc[i,"DPC期間"] == "期間I":\n        ddpc.loc[i,"DPC入院料"] = ddpc.loc[i,"入院期間Ⅰ（点）"]\n    elif ddpc.loc[i,"DPC期間"] == "期間II":\n        ddpc.loc[i,"DPC入院料"] = ddpc.loc[i,"入院期間Ⅱ（点）"]\n    elif ddpc.loc[i,"DPC期間"] == "期間III":\n        ddpc.loc[i,"DPC入院料"] = ddpc.loc[i,"入院期間Ⅲ（点）"]\n    else:\n        pass\n'

In [159]:
for i in dpc.index:
    if dpc.loc[i,"DPC期間"] == "期間I":
        dpc.loc[i,"DPC入院料"] = dpc.loc[i,"入院期間Ⅰ（点）"]
    elif dpc.loc[i,"DPC期間"] == "期間II":
        dpc.loc[i,"DPC入院料"] = dpc.loc[i,"入院期間Ⅱ（点）"]
    elif dpc.loc[i,"DPC期間"] == "期間III":
        dpc.loc[i,"DPC入院料"] = dpc.loc[i,"入院期間Ⅲ（点）"]
    else:
        pass

In [5]:
#dpc
# このデータを使えば実施年月日ごとに出来高点数とＤＰＣ入院料との差分を把握できる。

In [163]:
# jkeyで重複を削除（ＤＰＣ入院料レコード作成のため）
nyu = fl.drop_duplicates("jkey")

In [165]:
nyu.sort_values(["データ識別番号","実施年月日"],inplace=True)

C:\Program Files\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [166]:
nyu = nyu.reset_index()

In [167]:
nyu = nyu.drop("index",1)

In [168]:
# nyuに"在院日数"を付加
nyu["在院日数"] = nyu["実施年月日"] - nyu["入院年月日"] + datetime.timedelta(days=1)

In [170]:
dpc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2688 entries, 0 to 2687
Data columns (total 17 columns):
データ識別番号     2688 non-null int64
入院年月日       2688 non-null object
実施年月日       2688 non-null object
在院日数        2688 non-null timedelta64[ns]
key         2688 non-null object
診断群分類番号     2688 non-null object
入院期間Ⅰ（日）    2688 non-null timedelta64[ns]
入院期間Ⅱ（日）    2688 non-null timedelta64[ns]
入院期間Ⅲ（日）    2688 non-null timedelta64[ns]
入院期間Ⅰ（点）    2688 non-null float64
入院期間Ⅱ（点）    2688 non-null float64
入院期間Ⅲ（点）    2688 non-null float64
jkey        2688 non-null object
日別出来高総点数    2688 non-null float64
DPC期間       2688 non-null object
dpc総点数      2688 non-null float64
DPC入院料      2688 non-null object
dtypes: float64(5), int64(1), object(7), timedelta64[ns](4)
memory usage: 283.5+ KB


In [171]:
nyu.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8127 entries, 0 to 8126
Data columns (total 33 columns):
施設コード         8127 non-null object
データ識別番号       8127 non-null int64
退院年月日         8127 non-null object
入院年月日         8127 non-null object
データ区分         8127 non-null float64
順序番号          8127 non-null float64
病院点数マスタコード    8127 non-null float64
レセプト電算コード     8127 non-null float64
解釈番号          8127 non-null object
診療明細名称        8127 non-null object
行為点数          8127 non-null float64
行為薬剤料         8127 non-null float64
行為材料料         8127 non-null float64
円点区分          8127 non-null float64
行為回数          8127 non-null float64
保険者番号         8102 non-null float64
レセプト種別コード     8102 non-null float64
実施年月日         8127 non-null object
レセプト科区分       8102 non-null float64
診療科区分         8127 non-null object
医師コード         8127 non-null object
病棟コード         8127 non-null object
病棟区分          8127 non-null float64
入外区分          8102 non-null object
施設タイプ         0 non-null object
医療機関係数   

In [145]:
#nyu.reset_index(inplace=True)

In [146]:
#dpc.reset_index(inplace=True)

In [175]:
nyu["在院日数"].dt.days[0]

240

In [176]:
nyu.loc[0,"データ区分"]

21.0

In [178]:
nyu.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8127 entries, 0 to 8126
Data columns (total 33 columns):
施設コード         8127 non-null object
データ識別番号       8127 non-null int64
退院年月日         8127 non-null object
入院年月日         8127 non-null object
データ区分         8127 non-null float64
順序番号          8127 non-null float64
病院点数マスタコード    8127 non-null float64
レセプト電算コード     8127 non-null float64
解釈番号          8127 non-null object
診療明細名称        8127 non-null object
行為点数          8127 non-null float64
行為薬剤料         8127 non-null float64
行為材料料         8127 non-null float64
円点区分          8127 non-null float64
行為回数          8127 non-null float64
保険者番号         8102 non-null float64
レセプト種別コード     8102 non-null float64
実施年月日         8127 non-null object
レセプト科区分       8102 non-null float64
診療科区分         8127 non-null object
医師コード         8127 non-null object
病棟コード         8127 non-null object
病棟区分          8127 non-null float64
入外区分          8102 non-null object
施設タイプ         0 non-null object
医療機関係数   

In [179]:
nyu["dnz_key"] = nyu["データ識別番号"].apply(lambda x:str(x))+"_"+nyu["入院年月日"].apply(lambda x:str(x))+"_"+nyu["在院日数"].dt.days.apply(lambda x:str(x))

In [180]:
dpc["dnz_key"] = dpc["データ識別番号"].apply(lambda x:str(x))+"_"+dpc["入院年月日"].apply(lambda x:str(x))+"_"+dpc["在院日数"].dt.days.apply(lambda x:str(x))

In [181]:
# nyuとdpcを使う。データ識別番号と入院年月日と在院日数で見て、合致していれば、nyuの各カラムを上書き：

# "データ区分" = 93,
# "順序番号" =　int("在院日数")
# "病院点数マスタコード" = 0
# "レセプト電算コード" = 930000000
# "解釈番号" =　""
# "診療明細名称" =　dpc."診断群分類番号"  ←あとでdpcmasterからテーブルを抜いてきて日本語名称（例：　貧血（その他）　手術あり　手術・処置等２　なし　に書き換え
# "行為点数" = dpc."DPC入院料"
# "行為薬剤料" =　0
# "行為材料料" = 0
# "円点区分" = 0
# "行為回数" = 1

def my93(dx,dy):
    for i in dx.index:
        if dx.loc[i,"dnz_key"] in dy["dnz_key"].tolist():
            item = dx.loc[i,"dnz_key"]
            j = dy[dy["dnz_key"]==item].index[0]
            dx.loc[i,"データ区分"] = 93
            dx.loc[i,"順序番号"] = dx["在院日数"].dt.days[i]
            dx.loc[i,"病院点数マスタコード"] = 0
            dx.loc[i,"レセプト電算コード"] = 930000000
            dx.loc[i,"解釈番号"] = ""
            dx.loc[i,"診療明細名称"] = dy.loc[j,"診断群分類番号"]
            dx.loc[i,"行為点数"] = dy.loc[j,"DPC入院料"]
            dx.loc[i,"行為薬剤料"] = 0
            dx.loc[i,"行為材料料"] = 0
            dx.loc[i,"円点区分"] = 0
            dx.loc[i,"行為回数"] = 1
        else:
            pass


In [182]:
dpc_short = dpc[["dnz_key","診断群分類番号","DPC入院料"]]

In [183]:
my93(nyu,dpc_short)

In [185]:
'''
# nyuとdpcを使う。データ識別番号と入院年月日と在院日数で見て、合致していれば、nyuの各カラムを上書き：

# "データ区分" = 93,
# "順序番号" =　int("在院日数")
# "病院点数マスタコード" = 0
# "レセプト電算コード" = 930000000
# "解釈番号" =　""
# "診療明細名称" =　dpc."診断群分類番号"  ←あとでdpcmasterからテーブルを抜いてきて日本語名称（例：　貧血（その他）　手術あり　手術・処置等２　なし　に書き換え
# "行為点数" = dpc."DPC入院料"
# "行為薬剤料" =　0
# "行為材料料" = 0
# "円点区分" = 0
# "行為回数" = 1

@numba.jit
def my93a(dx,dy):
    for i in dx.index:
        for j in dy.index:
            if dx.loc[i,"dnz_key"] == dy.loc[j,"dnz_key"]:
                dx.loc[i,"データ区分"] = 93
                dx.loc[i,"順序番号"] = dx["在院日数"].dt.days[i]
                dx.loc[i,"病院点数マスタコード"] = 0
                dx.loc[i,"レセプト電算コード"] = 930000000
                dx.loc[i,"解釈番号"] = ""
                #dx.loc[i,"診療明細名称"] = dy.loc[j,"診断群分類番号"]
                #dx.loc[i,"行為点数"] = dy.loc[j,"DPC入院料"]
                dx.loc[i,"行為薬剤料"] = 0
                dx.loc[i,"行為材料料"] = 0
                dx.loc[i,"円点区分"] = 0
                dx.loc[i,"行為回数"] = 1
            else:
                pass

@numba.jit
def my93b(dx,dy):
    for i in dx.index:
        for j in dy.index:
            if dx.loc[i,"dnz_key"] == dy.loc[j,"dnz_key"]:
                #dx.loc[i,"データ区分"] = 93
                #dx.loc[i,"順序番号"] = dx["在院日数"].dt.days[i]
                #dx.loc[i,"病院点数マスタコード"] = 0
                #dx.loc[i,"レセプト電算コード"] = 930000000
                #dx.loc[i,"解釈番号"] = ""
                dx.loc[i,"診療明細名称"] = dy.loc[j,"診断群分類番号"]
                dx.loc[i,"行為点数"] = dy.loc[j,"DPC入院料"]
                #dx.loc[i,"行為薬剤料"] = 0
                #dx.loc[i,"行為材料料"] = 0
                #dx.loc[i,"円点区分"] = 0
                #dx.loc[i,"行為回数"] = 1
            else:
                pass
''''''

In [1]:
#my93a(nyu,dpc)

In [ ]:
#my93b(nyu,dpc)

In [184]:
'''
# nyuとdpcを使う。データ識別番号と入院年月日と在院日数で見て、合致していれば、nyuの各カラムを上書き：

# "データ区分" = 93,
# "順序番号" =　int("在院日数")
# "病院点数マスタコード" = 0
# "レセプト電算コード" = 930000000
# "解釈番号" =　""
# "診療明細名称" =　dpc."診断群分類番号"  ←あとでdpcmasterからテーブルを抜いてきて日本語名称（例：　貧血（その他）　手術あり　手術・処置等２　なし　に書き換え
# "行為点数" = dpc."DPC入院料"
# "行為薬剤料" =　0
# "行為材料料" = 0
# "円点区分" = 0
# "行為回数" = 1

for i in nyu.index:
    for j in dpc.index:
        if nyu.loc[i,"dnz_key"] == dpc.loc[j,"dnz_key"]:
            nyu.loc[i,"データ区分"] = 93
            nyu.loc[i,"順序番号"] = nyu["在院日数"].dt.days[i]
            nyu.loc[i,"病院点数マスタコード"] = 0
            nyu.loc[i,"レセプト電算コード"] = 930000000
            nyu.loc[i,"解釈番号"] = ""
            nyu.loc[i,"診療明細名称"] = dpc.loc[j,"診断群分類番号"]
            nyu.loc[i,"行為点数"] = dpc.loc[j,"DPC入院料"]
            nyu.loc[i,"行為薬剤料"] = 0
            nyu.loc[i,"行為材料料"] = 0
            nyu.loc[i,"円点区分"] = 0
            nyu.loc[i,"行為回数"] = 1
        else:
            pass
'''

'\n# nyuとdpcを使う。データ識別番号と入院年月日と在院日数で見て、合致していれば、nyuの各カラムを上書き：\n\n# "データ区分" = 93,\n# "順序番号" =\u3000int("在院日数")\n# "病院点数マスタコード" = 0\n# "レセプト電算コード" = 930000000\n# "解釈番号" =\u3000""\n# "診療明細名称" =\u3000dpc."診断群分類番号"  ←あとでdpcmasterからテーブルを抜いてきて日本語名称（例：\u3000貧血（その他）\u3000手術あり\u3000手術・処置等２\u3000なし\u3000に書き換え\n# "行為点数" = dpc."DPC入院料"\n# "行為薬剤料" =\u30000\n# "行為材料料" = 0\n# "円点区分" = 0\n# "行為回数" = 1\n\nfor i in nyu.index:\n    for j in dpc.index:\n        if nyu.loc[i,"dnz_key"] == dpc.loc[j,"dnz_key"]:\n            nyu.loc[i,"データ区分"] = 93\n            nyu.loc[i,"順序番号"] = nyu["在院日数"].dt.days[i]\n            nyu.loc[i,"病院点数マスタコード"] = 0\n            nyu.loc[i,"レセプト電算コード"] = 930000000\n            nyu.loc[i,"解釈番号"] = ""\n            nyu.loc[i,"診療明細名称"] = dpc.loc[j,"診断群分類番号"]\n            nyu.loc[i,"行為点数"] = dpc.loc[j,"DPC入院料"]\n            nyu.loc[i,"行為薬剤料"] = 0\n            nyu.loc[i,"行為材料料"] = 0\n            nyu.loc[i,"円点区分"] = 0\n            nyu.loc[i,"行為回数"] = 1\n        else:\n            pass\n'

In [213]:
'''
# nyuとdpcを使う。データ識別番号と入院年月日と在院日数で見て、合致していれば、nyuの各カラムを上書き：

# "データ区分" = 93,
# "順序番号" =　int("在院日数")
# "病院点数マスタコード" = 0
# "レセプト電算コード" = 930000000
# "解釈番号" =　""
# "診療明細名称" =　dpc."診断群分類番号"  ←あとでdpcmasterからテーブルを抜いてきて日本語名称（例：　貧血（その他）　手術あり　手術・処置等２　なし　に書き換え
# "行為点数" = dpc."DPC入院料"
# "行為薬剤料" =　0
# "行為材料料" = 0
# "円点区分" = 0
# "行為回数" = 1

for i in nyu.index:
    for j in dpc.index:
        if nyu.loc[i,"データ識別番号"] == dpc.loc[j,"データ識別番号"]:
            if nyu.loc[i,"入院年月日"] == dpc.loc[j,"入院年月日"]:
                if nyu.loc[i,"在院日数"] == dpc.loc[j,"在院日数"]:
                    nyu.loc[i,"データ区分"] = 93
                    nyu.loc[i,"順序番号"] = nyu["在院日数"].dt.days[i]
                    nyu.loc[i,"病院点数マスタコード"] = 0
                    nyu.loc[i,"レセプト電算コード"] = 930000000
                    nyu.loc[i,"解釈番号"] = ""
                    nyu.loc[i,"診療明細名称"] = dpc.loc[j,"診断群分類番号"]
                    nyu.loc[i,"行為点数"] = dpc.loc[j,"DPC入院料"]
                    nyu.loc[i,"行為薬剤料"] = 0
                    nyu.loc[i,"行為材料料"] = 0
                    nyu.loc[i,"円点区分"] = 0
                    nyu.loc[i,"行為回数"] = 1
                else:
                    pass
            else:
                pass
        else:
            pass
'''

C:\Anaconda3\lib\site-packages\pandas\core\indexing.py:465: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [215]:
#nyu.to_excel("test_nyu.xlsx")

In [185]:
with pg.connect(database='uonuma_kikan',user='postgres',host='localhost',port=5432,password=12345) as conn:
    sql = """
SELECT 診断群分類番号
       ,傷病名
       ,手術名
       ,手術・処置等１
       ,手術・処置等２
       ,定義副傷病
       ,重症度等
  FROM dpcmaster_{0};""".format(nengo)
    mstr_name = psql.read_sql(sql,conn)

In [186]:
mstr_name = mstr_name.fillna("")

In [187]:
mstr_name.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4918 entries, 0 to 4917
Data columns (total 7 columns):
診断群分類番号    4918 non-null object
傷病名        4918 non-null object
手術名        4918 non-null object
手術・処置等１    4918 non-null object
手術・処置等２    4918 non-null object
定義副傷病      4918 non-null object
重症度等       4918 non-null object
dtypes: object(7)
memory usage: 134.5+ KB


In [188]:
set(mstr_name["手術・処置等１"])

{'',
 'あり',
 'なし',
 'なし、１,２あり',
 'なし、１,３あり',
 'なし、１あり',
 '１,２あり',
 '１あり',
 '２あり',
 '３あり',
 '５あり'}

In [189]:
mstr_name["手術・処置等１"] = mstr_name["手術・処置等１"].apply(lambda x:"手術・処置等１あり" if "あり" in x else x)
mstr_name["手術・処置等１"] = mstr_name["手術・処置等１"].apply(lambda x:"手術・処置等１なし" if x=="なし" else x)

In [190]:
set(mstr_name["手術・処置等２"])

{'', 'あり', 'なし', '１あり', '２あり', '３あり', '４あり', '５あり', '６あり', '７あり', '８あり', '９あり'}

In [191]:
mstr_name["手術・処置等２"] = mstr_name["手術・処置等２"].apply(lambda x:"手術・処置等２あり" if "あり" in x else x)
mstr_name["手術・処置等２"] = mstr_name["手術・処置等２"].apply(lambda x:"手術・処置等２なし" if x=='なし' else x)

In [192]:
set(mstr_name["定義副傷病"])

{'', 'あり', 'なし', '１あり', '２あり'}

In [193]:
mstr_name["定義副傷病"] = mstr_name["定義副傷病"].apply(lambda x:"副傷病あり" if "あり" in x else x)
mstr_name["定義副傷病"] = mstr_name["定義副傷病"].apply(lambda x:"副傷病なし" if x=="なし" else x)

In [194]:
# ＤＰＣ入院料の日本語名称を各列文字列結合して取得
mstr_name["診療明細名称"] = mstr_name["傷病名"]+"　"+mstr_name["手術名"]+"　"\
+mstr_name["手術・処置等１"]+"　"+mstr_name["手術・処置等２"]+"　"+mstr_name["定義副傷病"]+"　"+mstr_name["重症度等"]

In [195]:
# nyuの診療明細名称に入れた診断群分類番号を日本語名称に変更

#@numba.jit
def num2name(dx,dy):
    for i in dx.index:
        if dx.loc[i,"診療明細名称"] in dy["診断群分類番号"].tolist():
            item = dx.loc[i,"診療明細名称"]
            j = dy[dy["診断群分類番号"]==item].index[0]
            dx.loc[i,"診療明細名称"] = dy.loc[j,"診療明細名称"]
        else:
            pass

In [196]:
mstr_name_short = mstr_name[["診断群分類番号","診療明細名称"]]

In [197]:
num2name(nyu,mstr_name_short)

In [ ]:
'''
# nyuの診療明細名称に入れた診断群分類番号を日本語名称に変更

#@numba.jit
def num2name(dx,dy):
    for i in dx.index:
        for j in dy.index:
            if dx.loc[i,"診療明細名称"] == dy.loc[j,"診断群分類番号"]:
                dx.loc[i,"診療明細名称"] = dy.loc[j,"診療明細名称"]
            else:
                pass
'''

In [ ]:
#num2name(nyu,mstr_name)

In [ ]:
'''
# nyuの診療明細名称に入れた診断群分類番号を日本語名称に変更

@numba.jit
for i in nyu.index:
    for j in mstr_name.index:
        if nyu.loc[i,"診療明細名称"] == mstr_name.loc[j,"診断群分類番号"]:
            nyu.loc[i,"診療明細名称"] = mstr_name.loc[j,"診療明細名称"]
        else:
            pass
'''

In [199]:
nyu.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8127 entries, 0 to 8126
Data columns (total 34 columns):
施設コード         8127 non-null object
データ識別番号       8127 non-null int64
退院年月日         8127 non-null object
入院年月日         8127 non-null object
データ区分         8127 non-null float64
順序番号          8127 non-null float64
病院点数マスタコード    8127 non-null float64
レセプト電算コード     8127 non-null float64
解釈番号          8127 non-null object
診療明細名称        8127 non-null object
行為点数          8127 non-null float64
行為薬剤料         8127 non-null float64
行為材料料         8127 non-null float64
円点区分          8127 non-null float64
行為回数          8127 non-null float64
保険者番号         8102 non-null float64
レセプト種別コード     8102 non-null float64
実施年月日         8127 non-null object
レセプト科区分       8102 non-null float64
診療科区分         8127 non-null object
医師コード         8127 non-null object
病棟コード         8127 non-null object
病棟区分          8127 non-null float64
入外区分          8102 non-null object
施設タイプ         0 non-null object
医療機関係数   

In [200]:
# ここでnyuから不要なカラムをドロップ
nyu.drop(["dnz_key"],1,inplace=True)

In [201]:
# ここでnyuから不要なカラムをドロップ
nyu.drop(["在院日数"],1,inplace=True)

In [202]:
print(EetBbb.shape, nyu.shape)

(230858, 32) (8127, 32)


In [203]:
#　ここでレセプト電算コード930000000となるレコードnyuとEetBbbをconcat

EetBbb = pd.concat([EetBbb,nyu])

In [204]:
EetBbb.shape

(238985, 32)

In [205]:
EetBbb["医療機関係数"] = cof

In [207]:
D = pd.DataFrame(EetBbb,columns=dcol)

In [208]:
D.reset_index(inplace=True)

In [209]:
D.drop(["index"],1,inplace=True)

In [210]:
D.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 238985 entries, 0 to 238984
Data columns (total 30 columns):
施設コード         238985 non-null object
データ識別番号       238985 non-null int64
退院年月日         238985 non-null object
入院年月日         238985 non-null object
データ区分         238985 non-null float64
順序番号          238985 non-null float64
病院点数マスタコード    238985 non-null float64
レセプト電算コード     238985 non-null float64
解釈番号          238985 non-null object
診療明細名称        238985 non-null object
行為点数          238985 non-null float64
行為薬剤料         238985 non-null float64
行為材料料         238985 non-null float64
円点区分          238985 non-null float64
行為回数          238985 non-null float64
保険者番号         80698 non-null float64
レセプト種別コード     80698 non-null float64
実施年月日         238985 non-null object
レセプト科区分       80698 non-null float64
診療科区分         238985 non-null object
医師コード         238985 non-null object
病棟コード         238985 non-null object
病棟区分          238985 non-null float64
入外区分          80698 non-null 

In [211]:
D["データ識別番号"] = D["データ識別番号"].apply(lambda x:str(x).zfill(10))

In [212]:
D["退院年月日"] = D["退院年月日"].apply(lambda x:int(x)).apply(lambda x:str(x))

In [213]:
D["入院年月日"] = D["入院年月日"].apply(lambda x:str(x).replace('-','')).apply(lambda x:int(x)).apply(lambda x:str(x))

In [214]:
D["実施年月日"] = D["実施年月日"].apply(lambda x:str(x).replace('-','')).apply(lambda x:int(x)).apply(lambda x:str(x))

In [215]:
#　医療機関係数以外float64になっているものはintに変更

In [216]:
flist = ["データ区分",        
"順序番号",         
"病院点数マスタコード",    
"レセプト電算コード",     
"行為点数",          
"行為薬剤料",         
"行為材料料",       
"円点区分",          
"行為回数",          
"保険者番号",         
"レセプト種別コード",     
"レセプト科区分",      
"病棟区分"]

In [217]:
int(D["病棟区分"][0])

1

In [218]:
D_f = D[flist].fillna("")

In [219]:
D_f_int = D_f.applymap(lambda x:int(x) if x!="" else x)

In [220]:
D[flist] = D_f_int

In [242]:
'''
#@numba.jit
for col in flist:
    D[col] = D[col].fillna("")
    for i in D.index:
        if D.loc[i,col] != "":
            D.loc[i,col] = int(D.loc[i,col])
        else:
            pass
'''

'\n#@numba.jit\nfor col in flist:\n    D[col] = D[col].fillna("")\n    for i in D.index:\n        if D.loc[i,col] != "":\n            D.loc[i,col] = int(D.loc[i,col])\n        else:\n            pass\n'

In [222]:
D.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 238985 entries, 0 to 238984
Data columns (total 30 columns):
施設コード         238985 non-null object
データ識別番号       238985 non-null object
退院年月日         238985 non-null object
入院年月日         238985 non-null object
データ区分         238985 non-null int64
順序番号          238985 non-null int64
病院点数マスタコード    238985 non-null int64
レセプト電算コード     238985 non-null int64
解釈番号          238985 non-null object
診療明細名称        238985 non-null object
行為点数          238985 non-null int64
行為薬剤料         238985 non-null int64
行為材料料         238985 non-null int64
円点区分          238985 non-null int64
行為回数          238985 non-null int64
保険者番号         238985 non-null object
レセプト種別コード     238985 non-null object
実施年月日         238985 non-null object
レセプト科区分       238985 non-null object
診療科区分         238985 non-null object
医師コード         238985 non-null object
病棟コード         238985 non-null object
病棟区分          238985 non-null int64
入外区分          80698 non-null object
施設タイプ       

In [211]:
#D["病棟区分"] = D["病棟区分"].astype(int)

In [212]:
#D[flist]

In [223]:
set(D["退院年月日"])

{'0',
 '20170101',
 '20170102',
 '20170103',
 '20170104',
 '20170105',
 '20170106',
 '20170107',
 '20170108',
 '20170109',
 '20170110',
 '20170111',
 '20170112',
 '20170113',
 '20170114',
 '20170115',
 '20170116',
 '20170117',
 '20170118',
 '20170119',
 '20170120',
 '20170121',
 '20170122',
 '20170123',
 '20170124',
 '20170125',
 '20170126',
 '20170127',
 '20170128',
 '20170129',
 '20170130',
 '20170131'}

In [224]:
D.to_csv("Dn_152410463_{0}.txt".format(month),sep='\t',index=False)